In [2]:
import json
import pandas as pd
import os
from psycopg2 import connect
import plotly_express as px

In [3]:
import requests

In [4]:
myconnection=connect(host='127.0.0.1',user='postgres',password='yusuff@12345',port='5432',database='phonepe')
cursor=myconnection.cursor()

cursor.execute("SELECT * FROM agg_transaction")
myconnection.commit()
table1=cursor.fetchall()
agg_trans=pd.DataFrame(table1,columns=["State","Quarter","Transaction Type","Transaction amount","Transaction count","Year"])

cursor.execute("SELECT * FROM agg_user")
myconnection.commit()
table2=cursor.fetchall()
agg_user=pd.DataFrame(table2,columns=["Mobile Brand","Transaction count","Percentage","Quarter","State","Year"])

cursor.execute("SELECT * FROM map_transaction")
myconnection.commit()
table3=cursor.fetchall()
map_trans=pd.DataFrame(table3,columns=["City","Transaction count","Transaction amount","Quarter","Year","State"])

cursor.execute("SELECT * FROM map_user")
myconnection.commit()
table4=cursor.fetchall()
map_user=pd.DataFrame(table4,columns=["District","Registered Users","App Opens","Quarter","Year","State"])

cursor.execute("SELECT * FROM top_transaction")
myconnection.commit()
table5=cursor.fetchall()
top_trans=pd.DataFrame(table5,columns=["State","Transaction Type","Transaction count","Transaction amount","Quarter","Year"])

cursor.execute("SELECT * FROM top_user")
myconnection.commit()
table6=cursor.fetchall()
top_user=pd.DataFrame(table6,columns=["District","Registered Users","Quarter","Year","State"])

In [5]:
def trans_amount_count(df,year):
    trans_y=df[df['Year']==year]
    trans_y.reset_index(drop=True,inplace=True)
    trans_g=trans_y.groupby("State")[['Transaction count','Transaction amount']].sum()
    trans_g.reset_index(inplace=True)
    figure_trans_count=px.bar(trans_g, x="State" , y="Transaction count",title=f"{year} Transaction count",height=600,width=750,color_discrete_sequence=px.colors.sequential.Aggrnyl)
    figure_trans_amount=px.bar(trans_g,x='State',y="Transaction amount",title=f"{year} Transaction amount",height=600,width=750,color_discrete_sequence=px.colors.sequential.Bluered_r)
    figure_trans_count.show()
    figure_trans_amount.show()
    url='https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson'
    response=requests.get(url)
    data1=json.loads(response.content)
    state_name=[]
    for i in data1["features"]:
        state_name.append(i['properties']['ST_NM'])
    state_name.sort()
    fig_india=px.choropleth(trans_g,geojson=data1,locations=state_name,featureidkey='properties.ST_NM',color='Transaction amount',color_continuous_scale="twilight",range_color=(trans_g['Transaction amount'].min(),trans_g['Transaction amount'].max()),hover_name='State',
                            title=f"{year} Transaction Amount",fitbounds="locations",height=600,width=600)
    fig_india.update_geos(visible=False)
    fig_india.show()
    fig_india1=px.choropleth(trans_g,geojson=data1,locations=state_name,featureidkey='properties.ST_NM',color='Transaction count',color_continuous_scale="twilight",range_color=(trans_g['Transaction count'].min(),trans_g['Transaction count'].max()),hover_name='State',
                            title=f"{year} Transaction count",fitbounds="locations",height=600,width=600)
    fig_india1.update_geos(visible=False)
    fig_india1.show()
    return trans_y

In [6]:
def trans_amount_count_quarter(df,quarter):
    trans_q=df[df['Quarter']==quarter]
    trans_q.reset_index(drop=True,inplace=True)
    trans_g=trans_q.groupby("State")[['Transaction count','Transaction amount']].sum()
    trans_g.reset_index(inplace=True)
    figure_trans_count=px.bar(trans_g, x="State" , y="Transaction count",title=f"Quarter {quarter} of Year {df['Year'].unique()} Transaction count",height=600,width=750,color_discrete_sequence=px.colors.sequential.Aggrnyl)
    figure_trans_amount=px.bar(trans_g,x='State',y="Transaction amount",title=f"Quarter {quarter} of Year {df['Year'].unique()} Transaction amount",height=600,width=750,color_discrete_sequence=px.colors.sequential.Bluered_r)
    figure_trans_count.show()
    figure_trans_amount.show()
    url='https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson'
    response=requests.get(url)
    data1=json.loads(response.content)
    state_name=[]
    for i in data1["features"]:
        state_name.append(i['properties']['ST_NM'])
    state_name.sort()
    fig_india=px.choropleth(trans_g,geojson=data1,locations=state_name,featureidkey='properties.ST_NM',color='Transaction amount',color_continuous_scale="twilight",range_color=(trans_g['Transaction amount'].min(),trans_g['Transaction amount'].max()),hover_name='State',
                            title=f"Quarter {quarter} of Year {df['Year'].unique()} Transaction Amount",fitbounds="locations",height=600,width=600)
    fig_india.update_geos(visible=False)
    fig_india.show()
    fig_india1=px.choropleth(trans_g,geojson=data1,locations=state_name,featureidkey='properties.ST_NM',color='Transaction count',color_continuous_scale="twilight",range_color=(trans_g['Transaction count'].min(),trans_g['Transaction count'].max()),hover_name='State',
                            title=f"Quarter {quarter} of Year {df['Year'].unique()} Transaction count",fitbounds="locations",height=600,width=600)
    fig_india1.update_geos(visible=False)
    fig_india1.show()
    return trans_q

In [ ]:
#trans_type
trans_q=trans_year[trans_year['State']=="West Bengal"]
trans_q.reset_index(drop=True,inplace=True)

trans_g=trans_q.groupby("Transaction Type")[['Transaction amount',"Transaction count"]].sum()
trans_g.reset_index(inplace=True)
trans_q
fig_trans_amount_pie=px.pie(data_frame=trans_g,names='Transaction Type',values='Transaction amount',width=800,title="Transaction amount",color="Transaction amount"
                            ,hole=0.2)
fig_trans_amount_pie.show()
fig_trans_count_pie=px.pie(data_frame=trans_g,names='Transaction Type',values='Transaction count',width=800,title="Transaction count",color="Transaction count",hole=0.2)
fig_trans_count_pie.show()

In [11]:
def user_transbr_fig(df,year):
    user_y=df[df['Year']==year]
    user_y.reset_index(drop=True,inplace=True)
    users_g=user_y.groupby('Mobile Brand')[['Transaction count']].sum()
    users_g.reset_index(inplace=True)
    figure_user_count=px.bar(users_g, x="Mobile Brand" , y="Transaction count",title=f"{year} Brand and Transaction count",height=600,width=750,color_discrete_sequence=px.colors.sequential.Aggrnyl,hover_name="Mobile Brand")
    figure_user_count.show()

In [13]:
def user_brtrans_count(df,quarter):
    user_y=df[df['Quarter']==quarter]
    user_y.reset_index(drop=True,inplace=True)
    users_g=user_y.groupby("Mobile Brand")[['Transaction count']].sum()
    users_g.reset_index(inplace=True)
    figure_trans_count=px.bar(users_g, x="Mobile Brand" , y="Transaction count",title=f" Brands and Transaction count",
                              height=600,width=750,color_discrete_sequence=px.colors.sequential.Aggrnyl,hover_name='Mobile Brand')
    figure_trans_count.show()

In [14]:
def user_transbr_fig(df,year,state,quarter):
    agg_state=df[df['State']==state]
    agg_state.reset_index(drop=True,inplace=True)
    agg_year=agg_state[agg_state['Year']==year]
    agg_year.reset_index(drop=True,inplace=True)
    agg_quat=agg_year[agg_year['Quarter']==quarter]
    agg_quat.reset_index(drop=True,inplace=True)
    agg_quat_g=agg_quat.groupby("Mobile Brand")[['Transaction count']].sum()
    agg_quat_g.reset_index(inplace=True)
    fig_quat=px.bar(agg_quat_g,x="Mobile Brand",y="Transaction count",title=f"{state}'s user transaction count in quarter {quarter} of year {year} figure"
                    ,hover_name="Mobile Brand",color_discrete_sequence=px.colors.sequential.Aggrnyl)
    fig_quat.show()

In [ ]:
#def user_transbr_fig(df,year,state,quarter):
agg_state=agg_user[agg_user['State']=="Karnataka"]
agg_state.reset_index(drop=True,inplace=True)
agg_year=agg_state[agg_state['Year']==2019]
agg_year.reset_index(drop=True,inplace=True)
agg_quat=agg_year[agg_year['Quarter']==3]
agg_quat.reset_index(drop=True,inplace=True)
agg_quat_g=agg_quat.groupby("Mobile Brand")[['Transaction count']].sum()
agg_quat_g.reset_index(inplace=True)
fig_quat=px.bar(agg_quat_g,x="Mobile Brand",y="Transaction count",title=f" user transaction count in quarter  of year  figure"
                    ,hover_name="Mobile Brand",color_discrete_sequence=px.colors.sequential.Aggrnyl)
fig_quat.show()

In [16]:
def user_transbr_fig(df,state,year,quarter):
    user_y=df[df['Year']==year]
    user_y.reset_index(drop=True,inplace=True)
    users_g=user_y.groupby('Mobile Brand')[['Transaction count']].sum()
    users_g.reset_index(inplace=True)
    figure_user_count=px.bar(users_g, x="Mobile Brand" , y="Transaction count",title=f"{year} Brand and Transaction count",height=600,width=750,color_discrete_sequence=px.colors.sequential.Aggrnyl,hover_name="Mobile Brand")
    figure_user_count.show()

In [17]:
##map transaction
def map_district(df,state):
    trans_q=df[df['State']==state]
    trans_q.reset_index(drop=True,inplace=True)

    trans_q=trans_q.groupby("City")[['Transaction amount',"Transaction count"]].sum()
    trans_q.reset_index(inplace=True)
    trans_q
    fig_trans_amount_bar=px.bar(trans_q,x='Transaction amount',y='City',width=800,title=f"{state} Transaction amount",color="Transaction amount"
                                )
    fig_trans_amount_bar.show()
    fig_trans_count_bar1=px.bar(trans_q,x='Transaction count',y='City',width=800,title=f"{state} Transaction count",color="Transaction count")
    fig_trans_count_bar1.show()

In [ ]:
#MAp user

user_y=map_user[map_user['Year']==2020]
user_y.reset_index(drop=True,inplace=True)
state=user_y[user_y['State']=='West Bengal']
users_g=state.groupby('District')[['Registered Users']].sum()
users_g.reset_index(inplace=True)
figure_user_count=px.bar(users_g, x="District" , y="Registered Users",title=f"Brand and Transaction count",height=600,width=750,color_discrete_sequence=px.colors.sequential.Aggrnyl,hover_name="District")
figure_user_count.show()

#map_user

In [24]:
#top transaction
states1=top_trans[top_trans['State']=="West Bengal"]
states1.reset_index(drop=True,inplace=True)
year=states1[states1['Year']==2020]
quarter1=year[year['Quarter']==3]
#trans_type=quarter1[quarter1["Transaction Type"]=="Recharge & bill payments"]
quarter1.reset_index(drop=True,inplace=True)
trans_type_g=quarter1.groupby("State")[["Transaction count"]].sum()
trans_type_g.reset_index(drop=True,inplace=True)
#top_fig_trans=px.bar(trans_type_g,x="State",y="Transaction count") 

In [ ]:
top_user_y=top_user[top_user['Year']==2020]
top_user_y.reset_index(drop=True,inplace=True)
top_state=top_user_y[top_user_y['State']=='West Bengal']
top_users_g=top_state.groupby('District')[['Registered Users']].sum()
top_users_g.reset_index(inplace=True)
figure_top_user_count=px.bar(top_users_g, x="District" , y="Registered Users",title=f"District and Transaction count",height=600,width=750,color_discrete_sequence=px.colors.sequential.Aggrnyl,hover_name="District")
figure_top_user_count.show()


In [ ]:
top_user_y=top_user[top_user['Year']==2020]
top_user_y.reset_index(drop=True,inplace=True)
top_user_g=top_user_y.groupby("State")['Registered Users'].sum()
df_g=pd.DataFrame(top_user_g)
df_g.reset_index()
url='https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson'
response=requests.get(url)
data1=json.loads(response.content)
state_name=[]
for i in data1["features"]:
    state_name.append(i['properties']['ST_NM'])
state_name.sort()
fig_india=px.choropleth(df_g,geojson=data1,locations=state_name,featureidkey='properties.ST_NM',color='Registered Users',color_continuous_scale="twilight",range_color=(df_g['Registered Users'].min(),df_g['Registered Users'].max()),
                        title=f"Quarter  of Year  Transaction Amount",fitbounds="locations",height=600,width=600)
fig_india.update_geos(visible=False)
fig_india.show()
